<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/experiment_llama2_chain_filter_flashrerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install langchain chromadb --quiet
%pip -q install sentence_transformers --quiet
%pip -q install -U FlagEmbedding --quiet
%pip install huggingface_hub --quiet
%pip install -q -U peft accelerate optimum --quiet
%pip install transformers==4.37.2 --quiet # downgraiding needed to solve AttributeError: 'LlamaRotaryEmbedding' object has no attribute 'cos_cached'
%pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --quiet
%pip install GPUtil --quiet
%pip install unstructured --quiet
%pip install --upgrade langsmith langchainhub --quiet
%pip install jq --quiet
%pip install tqdm --quiet
%pip install numpy==1.24.4 --quiet
%pip install --upgrade --quiet  flashrank

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import JSONLoader

import chromadb

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore
from langchain.document_loaders import TextLoader

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from operator import itemgetter

from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import PromptTemplate
import torch
import GPUtil
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

BGE models on the HuggingFace are one of the best open-source embedding models. BGE model is created by the Beijing Academy of Artificial Intelligence (BAAI). BAAI is a private non-profit organization engaged in AI research and development.

In [ ]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'}, #gpu
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Define the metadata extraction function.
def metadata_function(record: dict, metadata: dict) -> dict:

    metadata["cik"] = record.get("cik")
    metadata["company"] = record.get("company")
    metadata["filing_type"] = record.get("filing_type")
    metadata["filing_date"] = record.get("filing_date")
    metadata["period_of_report"] = record.get("period_of_report")
    metadata["state_location"] = record.get("state_location")
    metadata["fiscal_year_end"] = record.get("fiscal_year_end")
    metadata["htm_filing_link"] = record.get("htm_filing_link")
    metadata["filename"] = record.get("filename")

    return metadata

### Questions Creation:

In [ ]:
questions = pd.read_fwf("https://raw.githubusercontent.com/winterForestStump/thesis/main/questions/questions_ver2.txt", names=['question'])
#questions = pd.read_csv("https://raw.githubusercontent.com/winterForestStump/financebench/main/financebench_sample_150.csv")
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  35 non-null     object
dtypes: object(1)
memory usage: 408.0+ bytes


In [ ]:
# Here you need to enter the company name
company = "ADOBE SYSTEMS INC"
questions['question'] = questions['question'].str.replace('company', company)

In [ ]:
questions['question'][0]

'What is the total revenue generated by the ADOBE SYSTEMS INC and how has the revenue changed over the past few years?'

When splitting documents for retrieval, there are often conflicting desires:

You may want to have small documents, so that their embeddings can most accurately reflect their meaning. If too long, then the embeddings can lose meaning.
You want to have long enough documents that the context of each chunk is retained.
The ParentDocumentRetriever strikes that balance by splitting and storing small chunks of data. During retrieval, it first fetches the small chunks but then looks up the parent ids for those chunks and returns those larger documents.

In [ ]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports")

fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)

# This text splitter is used to create the parent documents - The big chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# This text splitter is used to create the child documents - The small chunks
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb')

vectorstore.persist()

In [ ]:
# Number of parent chunks retrieved
NUM_PAR_CHUNKS = 6

big_chunks_retriever = ParentDocumentRetriever(
    # The underlying vectorstore to use to store small chunks and their embedding vectors
    vectorstore=vectorstore,
    # The storage interface for the parent documents
    docstore=store,
    # The text splitter to use to create child documents.
    child_splitter=child_splitter,
    # The text splitter to use to create parent documents.
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company}, 'k': NUM_PAR_CHUNKS}
)

# by default the search_type is 'similarity, also 'mmr' and 'similarity_score_threshold' are available

Using a reranker to reduce the number of chunks from 6 to 2

In [ ]:
N_DOCS_RETURN = 2

compressor = FlashrankRerank(top_n = N_DOCS_RETURN)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=big_chunks_retriever
)

In [ ]:
all = vectorstore.get()['metadatas']
doc_ids = []
ciks = []
for i in range(len(all)):
  doc_ids.append(all[i]['doc_id'])
  ciks.append(all[i]['cik'])

print(f"The number of unique companies: {len(set(ciks))}")
print(f"The number of parent documents: {len(set(doc_ids))}")
print(f"The number of child documnets: {len(vectorstore.get()['documents'])}")

The number of unique companies: 32
The number of parent documents: 17561
The number of child documnets: 356275


In [ ]:
all[2]['company']

'ADOBE SYSTEMS INC'

### Creating the Pipeline:

Explanation of the parameters from the `generation_config`:
* `max_new_tokens`: the maximum number of tokens that can be generated in the output.
* `temperature`: controls how much randomness is introduced into the sampling process. A lower temperature value (closer to 0) makes the model more confident in its choices, resulting in less random outputs, while a higher temperature value (closer to 1) encourages more randomness and diversity.
* `top_p` controls nucleus sampling, a technique that considers only the most probable tokens with a cumulative probability above the threshold top_p. It helps in generating text that is both diverse and coherent, avoiding the inclusion of very low-probability tokens that could make the text nonsensical.
* `top_k` sampling limits the sampling pool to the k most likely next tokens. This further refines the set of tokens that the model will consider for generating the next piece of text, ensuring that the outputs remain relevant and coherent.
* `repetition_penalty` discourages the model from repeating the same tokens or phrases, promoting more interesting and diverse text. A value greater than 1 penalizes and thus reduces, the likelihood of tokens that have already appeared.

In [ ]:
MODEL_NAME = "TheBloke/Llama-2-7b-Chat-GPTQ"
TEMPERATURE = 0.0001
MAX_NEW_TOKENS = 2048
TOP_P = 0.90
REPETITION_PENALTY = 1.10

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto")

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = MAX_NEW_TOKENS
generation_config.temperature = TEMPERATURE
generation_config.top_p = TOP_P
generation_config.do_sample = True
generation_config.repetition_penalty = REPETITION_PENALTY


text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": TEMPERATURE})

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### Set Up the Chat Prompt Template
Chat prompt template will be used to structure the interaction with the LLM. It includes placeholders for context and a question, which will be dynamically filled during the execution of the chain.

In [ ]:
template = """
<s>[INST] <<SYS>>
Use the following information from company annual reports and answer the question at the end.
If the answer is not contained in the provided information or if there is NO context at all, say "The answer is not in the context".
<</SYS>>

{context}

{question} [/INST]
"""

prompt = ChatPromptTemplate.from_template(template)


The following code defines a pipeline for a question-answering system with retrieval augmentation.

It starts by taking a question and uses it both directly as the question and as input to a base retrieval system to fetch relevant context.

The retrieved context and the original question are then passed through a `RunnablePassthrough` for subsequent use, maintaining the context intact for reference.

Finally, the response is generated by a primary question-answering model `llm`, which takes the formatted prompt, consisting of the context and the question, and produces an answer.

In [ ]:
chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter('question')}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context"), "question": itemgetter('question')}
)

print('GPU Usage:')
GPUtil.showUtilization()

GPU Usage:
| ID | GPU | MEM |
------------------
|  0 |  0% | 73% |


### Chain Invocation:
This invocation triggers the entire sequence of operations defined in the chain. The retriever searches for relevant information, which is then passed along with the question through the prompt and into the Hugging Face model. The model generates a response based on the inputs it receives.

In [ ]:
from tqdm import tqdm

results_list = []

for i in tqdm(range(len(questions))):
    response = chain.invoke({"question": questions['question'][i]})
    results_list.append(pd.DataFrame({
        'question': [response['question']],
        'response': [response['response'].split('[/INST]\n')[1]],
        'context': [response['context']]
    }))

results = pd.concat(results_list, ignore_index=True)

  0%|          | 0/35 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  3%|▎         | 1/35 [00:24<13:55, 24.57s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  6%|▌         | 2/35 [01:05<18:47, 34.17s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  9%|▊         | 3/35 [01:24<14:36, 27.39s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficienc

In [ ]:
results

,question,response,context
0,What is the total revenue generated by the ADO...,"According to the provided document, Adobe Syst...",[page_content='Digital Marketing-Our Digital M...
1,What is the ADOBE SYSTEMS INC's cost of goods ...,"According to the provided document, Adobe Syst...",[page_content='Digital Marketing-Our Digital M...
2,What is the ADOBE SYSTEMS INC's gross profit m...,The answer is not in the provided information....,[page_content='ADOBE SYSTEMS INCORPORATED\nNOT...
3,What are the ADOBE SYSTEMS INC's major operati...,"Based on the provided information, the major o...","[page_content='1,117\n14,297\nThereafter\n-\n3..."
4,What is the ADOBE SYSTEMS INC's operating inco...,"According to the provided information, Adobe S...","[page_content='Other liabilities\n97,404\n88,6..."
5,What is the ADOBE SYSTEMS INC's net income for...,"According to the provided document, Adobe Syst...","[page_content='Other liabilities\n97,404\n88,6..."
6,What is the ADOBE SYSTEMS INC's earnings per s...,Based on the information provided in the docum...,"[page_content='3,214\n5,194\nIncome taxes paya..."
7,What is the ADOBE SYSTEMS INC's cash flow gene...,The answer is in the context. According to the...,[page_content='The timing of the resolution of...
8,How much has the ADOBE SYSTEMS INC invested in...,The answer is not provided in the given docume...,"[page_content=""INDEX TO CONSOLIDATED FINANCIAL..."
9,What is the ADOBE SYSTEMS INC's total outstand...,The answer is in the context. According to the...,[page_content='All legal costs associated with...


In [ ]:
results.to_json('results_ADOBE_rerank_6-2_filter.json')